In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pdb
from scipy import stats
from tqdm.auto import tqdm
tqdm.pandas()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
d_PCs = {'AT1G23490': 'pCH1',
'AT1G07890': 'pCH2',
'AT4G13930': 'pCH3',
'AT1G15690': 'pCH4',
'AT5G11740': 'pCH5',
'AT1G76200': 'pCL1',
'AT2G33040': 'pCL2',
'AT4G34110': 'pCL3',
'AT1G22300': 'pCL4',
'AT3G44110': 'pCL5',
'AT2G44060': 'pCM1',
'AT4G40040': 'pCM2',
'AT3G15730': 'pCM3',
'AT3G13920': 'pCM4',
'AT3G55440': 'pCM5',
'35s':'35s',
'pnos':'PNOS',
'AT4G40030': np.nan,
'057_tubq3': np.nan}

In [12]:
bc_extract = lambda x : re.findall(r"AGGAATG[ATGC]{18}GTGAG", x)

def build_BA(align, fwd, rev):
    align_dict = pd.read_csv(align, sep='\t', header=None, names=["read_id", "promoter", "seq"])
    align_f = pd.read_csv(fwd, sep='\t', header=None, names=["read_id", "readseq"])
    align_r = pd.read_csv(rev, sep='\t', header=None, names=["read_id", "readseq"])
    d_align = align_dict.set_index('read_id').promoter.to_dict()
    combined = pd.concat([align_f, align_r])
    combined["promoter"] = combined.read_id.map(d_align)
    combined = combined.dropna()
    combined["bc"] = combined.readseq.apply(lambda x: bc_extract(x)[0][7:-5])
    combined_bc = combined[["bc", "promoter"]]
    deduped_bc = combined_bc[~combined_bc.duplicated()].reset_index(drop=True)
    return deduped_bc

def file_read(fname):
    df = pd.read_csv(fname, sep='\t', header=None, names=["read_id", "seq"])
    all_counts = df.shape[0]
    df['bc'] = df.seq.apply(lambda x: bc_extract(x)[0][7:-5])
    df['prom'] = df.bc.map(BA_dict)
    #print('before:', df.shape)
    df = df.dropna()
    #print('after:', df.shape)
    freq = pd.DataFrame(df.bc.value_counts()[df.bc.value_counts() >= 5] / all_counts).reset_index().rename(columns={'index':'barcode', 'bc':'freq'})
    biorep, sample = fname.split('/')[1].split('.')[0].split('_')
    freq['biorep'] = biorep
    freq['sample'] = sample
    return freq

def file_read_input(fname):
    df = pd.read_csv(fname, sep='\t', header=None, names=["read_id", "seq"])
    all_counts = df.shape[0]
    df['bc'] = df.seq.apply(lambda x: bc_extract(x)[0][7:-5])
    df['prom'] = df.bc.map(BA_dict)
    #print('before:', df.shape)
    df = df.dropna()
    #print('after:', df.shape)
    freq = pd.DataFrame(df.bc.value_counts()[df.bc.value_counts() >= 5] / all_counts).reset_index().rename(columns={'index':'barcode', 'bc':'freq'})
    biorep, fluor, sample = fname.split('/')[1].split('.')[0].split('_')
    freq['biorep'] = biorep
    freq['fluor'] = fluor
    freq['sample'] = sample
    return freq

def map_quant_list(files_list):
    df_list = []
    for file in tqdm(files_list):
        df_list.append(file_read('tsvs/' + file))
    return(pd.concat(df_list))

def map_input_list(files_list):
    df_list = []
    for file in tqdm(files_list):
        df_list.append(file_read_input('tsvs/' + file))
    return(pd.concat(df_list))

def freq_input_map(input_df, barcode, biorep):
    d_input = input_df[input_df.biorep == biorep].set_index('barcode').freq.to_dict()
    try: 
        input_freq = d_input[barcode]
    except KeyError:
        input_freq = None
    return input_freq

def finishing_bootstrap(df):
    df = df.dropna().reset_index(drop=True)
    df['normalized_freq'] = df.freq/df.input_freq
    df['prom'] = df.barcode.map(BA_dict)
    df['pCONS'] = df.prom.map(d_PCs)
    df = df.dropna().reset_index(drop=True)
    return df

def graph_transform_mean(df):
    df = df.groupby(['pCONS','biorep','sample']).mean()[['normalized_freq']].reset_index()
    df['log_freq'] = df['normalized_freq'].apply(np.log2)
    return df

def graph_transform_median(df):
    df = df.groupby(['pCONS','biorep','sample']).median()[['normalized_freq']].reset_index()
    df['log_freq'] = df['normalized_freq'].apply(np.log2)
    return df

def swarm_func(df):
    order = graph_transform_mean(df).groupby('pCONS').mean()['log_freq'].sort_values().index
    sns.set(font_scale=1.4)
    sns.set_style('ticks')
    sns.set_palette('colorblind')
    fig, ax = plt.subplots(figsize=(20, 6)) 
    sns.swarmplot(data=graph_transform_mean(df), x='pCONS', y='log_freq', hue='biorep', order=order)


In [4]:
round_1 = build_BA('pacbio/new/pc_pacbio_alignment.tsv', 'pacbio/new/pc_bc_fwd.tsv', 'pacbio/new/pc_bc_rev.tsv')
round_2 = build_BA('pacbio/old/pc_pacbio_alignment.tsv', 'pacbio/old/pc_bc_fwd.tsv', 'pacbio/old/pc_bc_rev.tsv')

In [5]:
combined = pd.concat([round_1, round_2])
collapsed = combined.groupby('bc').agg(list).promoter.apply(np.unique)
unique_bc = combined.groupby('bc').agg(list).promoter.apply(np.unique).apply(len) == 1
BA_dict = collapsed[unique_bc].apply(lambda x: x[0]).to_dict()
#l_collide = list(combined.groupby('bc').agg(list)[combined.groupby('bc').agg(list).promoter.apply(len) != 1].index)

In [6]:
all_files = os.listdir('tsvs/')

tob_files = sorted([file for file in all_files if "T" in file])
let_files = sorted([file for file in all_files if "L" in file])

tob_in_files = sorted([file for file in tob_files if "I" in file])
tob_quant_files = sorted([file for file in tob_files if "I" not in file])

let_in_files = sorted([file for file in let_files if "I" in file])
let_quant_files = sorted([file for file in let_files if "I" not in file])

tob_in_green_files = sorted([file for file in tob_in_files if "GFP" in file])
tob_in_red_files = sorted([file for file in tob_in_files if "DSRed" in file])

tob_quant_green_files = sorted([file for file in tob_quant_files if "TG" in file])
tob_quant_red_files = sorted([file for file in tob_quant_files if "TR" in file])

let_in_green_files = sorted([file for file in let_in_files if "GFP" in file])
let_in_red_files = sorted([file for file in let_in_files if "DSRed" in file])

let_quant_green_files = sorted([file for file in let_quant_files if "LG" in file])
let_quant_red_files = sorted([file for file in let_quant_files if "LR" in file])

In [8]:
tob_quant_green = map_quant_list(tob_quant_green_files)
tob_quant_red = map_quant_list(tob_quant_red_files)

let_quant_green = map_quant_list(let_quant_green_files)
let_quant_red = map_quant_list(let_quant_red_files)

tob_input_green = map_input_list(tob_in_green_files)
tob_input_red = map_input_list(tob_in_red_files)

let_input_green = map_input_list(let_in_green_files)
let_input_red = map_input_list(let_in_red_files)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [9]:
tgi = tob_input_green.groupby(['barcode', 'biorep', "fluor"]).mean().reset_index()
tri = tob_input_red.groupby(['barcode', 'biorep', "fluor"]).mean().reset_index()

lgi = let_input_green.groupby(['barcode', 'biorep', "fluor"]).mean().reset_index()
lri = let_input_red.groupby(['barcode', 'biorep', "fluor"]).mean().reset_index()

In [10]:
tob_quant_green['input_freq'] = tob_quant_green.progress_apply(lambda x: freq_input_map(tgi, x['barcode'], x['biorep']), axis=1)
tob_quant_red['input_freq'] = tob_quant_red.progress_apply(lambda x: freq_input_map(tri, x['barcode'], x['biorep']), axis=1)

let_quant_green['input_freq'] = let_quant_green.progress_apply(lambda x: freq_input_map(lgi, x['barcode'], x['biorep']), axis=1)
let_quant_red['input_freq'] = let_quant_red.progress_apply(lambda x: freq_input_map(lri, x['barcode'], x['biorep']), axis=1)

  0%|          | 0/5898 [00:00<?, ?it/s]

  0%|          | 0/6401 [00:00<?, ?it/s]

  0%|          | 0/6075 [00:00<?, ?it/s]

  0%|          | 0/6355 [00:00<?, ?it/s]

In [11]:
tobacco_green = finishing_bootstrap(tob_quant_green)
tobacco_red = finishing_bootstrap(tob_quant_red)
lettuce_green = finishing_bootstrap(let_quant_green)
lettuce_red = finishing_bootstrap(let_quant_red)

In [13]:
# df = pd.concat(map(lambda x: graph_transform_mean(x), [tobacco_green, tobacco_red, lettuce_green, lettuce_red]))
# df = df.reset_index(drop=True)
# df['plant'] = df['sample'].apply(lambda x: 'Lettuce' if 'L' in x else 'Tobacco')
# df['fluor'] = df['sample'].apply(lambda x: 'GFP' if 'G' in x else 'DSRed')

df = pd.concat(map(lambda x: graph_transform_mean(x), [tobacco_green, tobacco_red, lettuce_green, lettuce_red]))
df = df.reset_index(drop=True)
df['plant'] = df['sample'].apply(lambda x: 'Lettuce' if 'L' in x else 'Tobacco')
df['fluor'] = df['sample'].apply(lambda x: 'GFP' if 'G' in x else 'DSRed')
df['tech'] = df['sample'].apply(lambda x: 'Tech2' if len(x) == 3 else 'Tech1')
df['ensemb_lab'] = df.pCONS.apply(lambda x: "pCL1-5" if "L" in x else "pCM1-5" if "M" in x else "pCH1-5" if "H" in x else x)

In [16]:
df.to_csv('tob_let_starr.csv', index=False)